In [1]:
import os

In [2]:
%pwd

'd:\\Data_science\\Projects\\End_to_End_ml_project_chest_CT_scan\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data_science\\Projects\\End_to_End_ml_project_chest_CT_scan'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    Params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

In [6]:
from chestCancerClassifier.constants import *
from chestCancerClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)


        create_directories([self.config.artifacts_root])
    

    def get_traning_config(self) -> TrainingConfig:
        traning = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.Unzip_dir, "chest_CT_scan_data")
        create_directories([
            Path(traning.root_dir)
        ])

        traning_config = TrainingConfig(
            root_dir=Path(traning.root_dir),
            trained_model_path=Path(traning.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            Params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
        )

        return traning_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
tf.config.run_functions_eagerly(True)

In [9]:
import tensorflow as tf
from pathlib import Path

class Training:
    def __init__(self, config):
        self.config = config
        self.model = None

    def get_base_model(self):
        # Load the model's architecture and weights.
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    # A dedicated method to compile the model.
    def compile_model(self):
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

    def train_valid_generator(self):
        # ... ( code for setting up generators) ...
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split= 0.20
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size= self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        if self.config.Params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator=valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_traning_config()
    training = Training(config=training_config)
    
    # 1. Load the model's architecture and weights.
    training.get_base_model() 
    
    # 2. Re-compile the model. This is the crucial step.
    training.compile_model() 
    
    # 3. Set up the data generators.
    training.train_valid_generator()
    
    # 4. Start the training process.
    training.train()
    
except Exception as e:
    raise e

[2025-09-21 21:26:34,713: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-21 21:26:34,726: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-21 21:26:34,729: INFO: common: created directory at: artifacts]
[2025-09-21 21:26:34,731: INFO: common: created directory at: artifacts\training]
[2025-09-21 21:26:35,038: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 178 images belonging to 4 classes.
Found 722 images belonging to 4 classes.


d:\Data_science\Projects\End_to_End_ml_project_chest_CT_scan\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


d:\Data_science\Projects\End_to_End_ml_project_chest_CT_scan\venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


45/45 ━━━━━━━━━━━━━━━━━━━━ 439s 10s/step - accuracy: 0.3059 - loss: 233.0045 - val_accuracy: 0.1761 - val_loss: 168.0481
Epoch 2/10
 1/45 ━━━━━━━━━━━━━━━━━━━━ 6:38 9s/step - accuracy: 0.2500 - loss: 106.6295

d:\Data_science\Projects\End_to_End_ml_project_chest_CT_scan\venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


45/45 ━━━━━━━━━━━━━━━━━━━━ 43s 764ms/step - accuracy: 0.2500 - loss: 106.6295 - val_accuracy: 0.2727 - val_loss: 153.8364
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 451s 10s/step - accuracy: 0.2875 - loss: 224.3512 - val_accuracy: 0.6307 - val_loss: 118.2124
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 43s 770ms/step - accuracy: 0.3125 - loss: 240.9144 - val_accuracy: 0.3920 - val_loss: 63.7645
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 444s 10s/step - accuracy: 0.3357 - loss: 171.4500 - val_accuracy: 0.3807 - val_loss: 135.6146
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 42s 757ms/step - accuracy: 0.4375 - loss: 132.6498 - val_accuracy: 0.4034 - val_loss: 118.0280
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 442s 10s/step - accuracy: 0.3938 - loss: 156.5532 - val_accuracy: 0.4375 - val_loss: 113.7126
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 43s 781ms/step - accuracy: 0.3750 - loss: 134.4650 - val_accuracy: 0.4830 - val_loss: 281.7186
Epoch 9/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 447s 10s/step - accuracy: 0.3994 - loss: 155.4